In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "True"

# Summarization

In [2]:
# %pip install --upgrade langchain_core

In [3]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader, TextLoader
from langchain_openai import ChatOpenAI

# Path to your .txt file
file_path = "/home/david/Documents/hackathon2024/splitData/one.txt"

# loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
loader = TextLoader(file_path)
docs = loader.load()
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(page_content=" No one works for emergency. Um, hello. Hello. Is everything okay? Um, my mommy and daddy. Uh-huh. I think there's a bullet on the floor. Is there the what? And there's a... There's a slide coming out of my dad's mouth and he fell off the bed. He did? Where's mommy at? I... I don't know. I think they're dead. What do you mean, sweetheart? I don't know. Okay, dude. Your daddy's on the floor. How old are you? I'm five years old and I have a dog in the house. Okay, there, there, okay. Let me get something right over to you. Did you go in your mommy and daddy's room? Oh, and there's a blood. Oh, over the place? Not all over. There's blood on the plant and blood on the floor. Oh, my goodness. You have your little doggy with you? And three cats. Well, get some cats too. Three cats and one dog. Okay. Are you the only one there? Is this like mommy and daddy? Oh, my... It's dad and mommy and daddy and they didn't even answer. Okay. Okay, what I want you to do, honey, I w

In [4]:
texts = docs[0].page_content
print(texts)

 No one works for emergency. Um, hello. Hello. Is everything okay? Um, my mommy and daddy. Uh-huh. I think there's a bullet on the floor. Is there the what? And there's a... There's a slide coming out of my dad's mouth and he fell off the bed. He did? Where's mommy at? I... I don't know. I think they're dead. What do you mean, sweetheart? I don't know. Okay, dude. Your daddy's on the floor. How old are you? I'm five years old and I have a dog in the house. Okay, there, there, okay. Let me get something right over to you. Did you go in your mommy and daddy's room? Oh, and there's a blood. Oh, over the place? Not all over. There's blood on the plant and blood on the floor. Oh, my goodness. You have your little doggy with you? And three cats. Well, get some cats too. Three cats and one dog. Okay. Are you the only one there? Is this like mommy and daddy? Oh, my... It's dad and mommy and daddy and they didn't even answer. Okay. Okay, what I want you to do, honey, I want you to stand up for 

In [5]:
from langchain.chains.summarize import load_summarize_chain
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
local_llm = "gpt-4-0125-preview"
llm = ChatOpenAI(
    model = local_llm,
    base_url = "http://10.128.138.175:11434/v1/",
    temperature=0,
    api_key="ollama",
    streaming=True,
)
chain = load_summarize_chain(llm, chain_type="stuff")

result = chain.invoke(docs)

print(result["output_text"])

A 5-year-old child calls 911 after finding their parents' bodies on the floor with a bullet nearby. The child is shaken and confused, but tries to provide information about what they saw. The 911 operator attempts to comfort the child and gather more details, including the names of the child's pets (Liza the dog and three cats).


In [6]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate

# Define prompt
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
# llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

docs = loader.load()
output = stuff_chain.invoke(docs)["output_text"]
print(output)

/home/david/Documents/gen-ui-python/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


A 5-year-old child calls 911 after finding their parents' bodies on the floor with a bullet nearby. The child is shaken and confused, but able to provide some details about the scene. The dispatcher tries to comfort the child and gather more information, including the names of the child's pets (Liza the dog and three cats). The child is eventually convinced to stand up and wait for help to arrive.


# Tagging

In [7]:
# Path to your model weights
local_model = "/home/david/Documents/cira-project/llmodel/ggufModel/model/llama-2-7b-chat-dpo.gguf"

In [8]:
# %pip uninstall langchain_community
# %pip install langchain_community
# %pip install --upgrade langchain_community

In [9]:
import multiprocessing

from langchain_community.chat_models import ChatLlamaCpp

llm_cpp = ChatLlamaCpp(
    temperature=0.5,
    model_path=local_model,
    n_ctx=10000,
    n_gpu_layers=8,
    n_batch=300,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    max_tokens=512,
    n_threads=multiprocessing.cpu_count() - 1,
    repeat_penalty=1.5,
    top_p=0.5,
    verbose=True,
)

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /home/david/Documents/cira-project/llmodel/ggufModel/model/llama-2-7b-chat-dpo.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'IncidentClassification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")

class IncidentClassification(BaseModel):
    violence: str = Field(description="Whether the incident involved violence")
    fire: str = Field(description="Whether the incident involved fire")
    medical: str = Field(description="Whether the incident involved medical attention")      
    suspicious_activity: str = Field(description="Whether the incident involved suspicious activity")                                                                                         


# LLM
local_llm = "gpt-4-0125-preview"
llm_original = ChatOpenAI(
    model = local_llm,
    base_url = "http://10.128.138.175:11434/v1/",
    temperature=0,
    api_key="ollama",
    streaming=True,
)
# llm = llm_original.with_structured_output(
#     IncidentClassification
# )

llm = llm_cpp.with_structured_output(
    IncidentClassification
)

tagging_chain = tagging_prompt | llm

In [11]:
output

"A 5-year-old child calls 911 after finding their parents' bodies on the floor with a bullet nearby. The child is shaken and confused, but able to provide some details about the scene. The dispatcher tries to comfort the child and gather more information, including the names of the child's pets (Liza the dog and three cats). The child is eventually convinced to stand up and wait for help to arrive."

In [12]:
# inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
res = tagging_chain.invoke({"input": output})
res

from_string grammar:
char ::= [^"\] | [\] char_1 
char_1 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
fire-kv ::= ["] [f] [i] [r] [e] ["] space [:] space string 
space ::= space_9 
string ::= ["] string_10 ["] space 
medical-kv ::= ["] [m] [e] [d] [i] [c] [a] [l] ["] space [:] space string 
root ::= [{] space violence-kv [,] space fire-kv [,] space medical-kv [,] space suspicious-activity-kv [}] space 
violence-kv ::= ["] [v] [i] [o] [l] [e] [n] [c] [e] ["] space [:] space string 
suspicious-activity-kv ::= ["] [s] [u] [s] [p] [i] [c] [i] [o] [u] [s] [_] [a] [c] [t] [i] [v] [i] [t] [y] ["] space [:] space string 
space_9 ::= [ ] | 
string_10 ::= char string_10 | 


llama_print_timings:        load time =    7056.23 ms
llama_print_timings:      sample time =     108.79 ms /    38 runs   (    2.86 ms per token,   349.31 tokens per second)
llama_print_timings: prompt eval time =    7055.41 ms /   135 tokens (   52.26 ms per token,    19.13 tokens per second)
llama

IncidentClassification(violence='yes', fire='no', medical='Yes(child injured)', suspicious_activity='No')

In [13]:
info_tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'IncidentInformation' function.

Passage:
{input}
"""
)

class IncidentInformation(BaseModel):
    who: str = Field(description="Who was involved in the incident")
    what: str = Field(description="What happened during the incident")
    when: str = Field(description="When did the incident occur")
    where: str = Field(description="Where did the incident occur")
    why: str = Field(description="Why did the incident occur")


info_llm = llm_cpp.with_structured_output(
    IncidentInformation
)

info_tagging_chain = info_tagging_prompt | info_llm

In [14]:
docs 

[Document(page_content=" No one works for emergency. Um, hello. Hello. Is everything okay? Um, my mommy and daddy. Uh-huh. I think there's a bullet on the floor. Is there the what? And there's a... There's a slide coming out of my dad's mouth and he fell off the bed. He did? Where's mommy at? I... I don't know. I think they're dead. What do you mean, sweetheart? I don't know. Okay, dude. Your daddy's on the floor. How old are you? I'm five years old and I have a dog in the house. Okay, there, there, okay. Let me get something right over to you. Did you go in your mommy and daddy's room? Oh, and there's a blood. Oh, over the place? Not all over. There's blood on the plant and blood on the floor. Oh, my goodness. You have your little doggy with you? And three cats. Well, get some cats too. Three cats and one dog. Okay. Are you the only one there? Is this like mommy and daddy? Oh, my... It's dad and mommy and daddy and they didn't even answer. Okay. Okay, what I want you to do, honey, I w

In [15]:
# inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
info_res = info_tagging_chain.invoke({"input": docs})
info_res

from_string grammar:
char ::= [^"\] | [\] char_1 
char_1 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
root ::= [{] space who-kv [,] space what-kv [,] space when-kv [,] space where-kv [,] space why-kv [}] space 
space ::= space_9 
who-kv ::= ["] [w] [h] [o] ["] space [:] space string 
what-kv ::= ["] [w] [h] [a] [t] ["] space [:] space string 
when-kv ::= ["] [w] [h] [e] [n] ["] space [:] space string 
where-kv ::= ["] [w] [h] [e] [r] [e] ["] space [:] space string 
why-kv ::= ["] [w] [h] [y] ["] space [:] space string 
space_9 ::= [ ] | 
string ::= ["] string_11 ["] space 
string_11 ::= char string_11 | 

Llama.generate: prefix-match hit

llama_print_timings:        load time =    7056.23 ms
llama_print_timings:      sample time =     253.77 ms /    81 runs   (    3.13 ms per token,   319.19 tokens per second)
llama_print_timings: prompt eval time =   36508.80 ms /   592 tokens (   61.67 ms per token,    16.22 tokens per second)
llama_print_timings:        eval

IncidentInformation(who='mommy and daddy', what='a bullet on the floor, a slide coming out of my dad’s mouth... blood on plant &amp; floors.', when='tonight', where=". I think they're dead.", why=".I don't know. Okay, dude....")

# Question Generation

In [16]:
# %pip install llama-index-llms-openai
# %pip install llama-index

In [17]:
import logging
import sys
import pandas as pd

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [18]:
# %pip install llama-index
# %pip install spacy
# %pip install llama-index-llms-langchain

In [19]:
from llama_index.core.evaluation import DatasetGenerator, RelevancyEvaluator
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response
from llama_index.llms.openai import OpenAI

In [20]:
# !mkdir -p 'data/paul_graham/'
# !wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

In [21]:
# reader = SimpleDirectoryReader("./data/paul_graham/")
reader = SimpleDirectoryReader("/home/david/Documents/hackathon2024/splitData/")
documents = reader.load_data()
documents

[Document(id_='6ff0663f-d046-4e45-8b78-905066d6eaf2', embedding=None, metadata={'file_path': '/home/david/Documents/hackathon2024/splitData/one.txt', 'file_name': 'one.txt', 'file_type': 'text/plain', 'file_size': 1736, 'creation_date': '2024-06-23', 'last_modified_date': '2024-06-23'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text=" No one works for emergency. Um, hello. Hello. Is everything okay? Um, my mommy and daddy. Uh-huh. I think there's a bullet on the floor. Is there the what? And there's a... There's a slide coming out of my dad's mouth and he fell off the bed. He did? Where's mommy at? I... I don't know. I think they're dead. What do you mean, sweetheart? I don't know. Okay, dude. Your daddy's on the floor. How old are you? I'm five years ol

In [22]:
llm_doc = ChatOpenAI(
    model = local_llm,
    base_url = "http://10.128.138.175:11434/v1/",
    temperature=0,
    api_key="ollama",
    streaming=True,
)

In [23]:
documents[0].text

" No one works for emergency. Um, hello. Hello. Is everything okay? Um, my mommy and daddy. Uh-huh. I think there's a bullet on the floor. Is there the what? And there's a... There's a slide coming out of my dad's mouth and he fell off the bed. He did? Where's mommy at? I... I don't know. I think they're dead. What do you mean, sweetheart? I don't know. Okay, dude. Your daddy's on the floor. How old are you? I'm five years old and I have a dog in the house. Okay, there, there, okay. Let me get something right over to you. Did you go in your mommy and daddy's room? Oh, and there's a blood. Oh, over the place? Not all over. There's blood on the plant and blood on the floor. Oh, my goodness. You have your little doggy with you? And three cats. Well, get some cats too. Three cats and one dog. Okay. Are you the only one there? Is this like mommy and daddy? Oh, my... It's dad and mommy and daddy and they didn't even answer. Okay. Okay, what I want you to do, honey, I want you to stand up for

In [24]:
data_generator = DatasetGenerator.from_documents(
    llm=llm_doc,
    documents=documents,
)

/home/david/Documents/gen-ui-python/.venv/lib/python3.10/site-packages/llama_index/core/evaluation/dataset_generation.py:215: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


In [25]:
data_generator

In [26]:
import nest_asyncio
nest_asyncio.apply()

In [27]:
eval_questions = data_generator.generate_questions_from_nodes(
    num=5,
)
eval_questions


/home/david/Documents/gen-ui-python/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


INFO:httpx:HTTP Request: POST http://10.128.138.175:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://10.128.138.175:11434/v1/chat/completions "HTTP/1.1 200 OK"


/home/david/Documents/gen-ui-python/.venv/lib/python3.10/site-packages/llama_index/core/evaluation/dataset_generation.py:312: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


['Here are 10 questions based on the given context information:',
 'What is the file path of the text file mentioned in the context?',
 'Answer: /home/david/Documents/hackathon20/24/splitData/one.txt',
 'What is the type of the file mentioned in the context?',
 'Answer: text/plain']

In [28]:
# %pip install llama_index.llms.fireworks

In [29]:
# evaluator_gpt4 = RelevancyEvaluator(llm=llm_doc)
# evaluator_gpt4

In [30]:
# # create vector index
# vector_index = VectorStoreIndex.from_documents(
#     # llm=llm_doc,
#     documents=documents
# )

In [31]:
# vector_index

In [32]:
# # define jupyter display function
# def display_eval_df(query: str, response: Response, eval_result: str) -> None:
#     eval_df = pd.DataFrame(
#         {
#             "Query": query,
#             "Response": str(response),
#             "Source": (
#                 response.source_nodes[0].node.get_content()[:1000] + "..."
#             ),
#             "Evaluation Result": eval_result,
#         },
#         index=[0],
#     )
#     eval_df = eval_df.style.set_properties(
#         **{
#             "inline-size": "600px",
#             "overflow-wrap": "break-word",
#         },
#         subset=["Response", "Source"]
#     )
#     display(eval_df)

In [33]:
# query_engine = vector_index.as_query_engine()
# response_vector = query_engine.query(eval_questions[1])
# eval_result = evaluator_gpt4.evaluate_response(
#     query=eval_questions[1], response=response_vector
# )

# Long-Context Reorder

reduce the important information degradation

In [34]:
# %pip install --upgrade --quiet  sentence-transformers langchain-chroma langchain langchain-openai > /dev/null

In [35]:
# from langchain.chains import LLMChain, StuffDocumentsChain
# from langchain_chroma import Chroma
# from langchain_community.document_transformers import (
#     LongContextReorder,
# )
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_core.prompts import PromptTemplate
# from langchain_openai import OpenAI

# # Get embeddings.
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # texts = [
# #     "Basquetball is a great sport.",
# #     "Fly me to the moon is one of my favourite songs.",
# #     "The Celtics are my favourite team.",
# #     "This is a document about the Boston Celtics",
# #     "I simply love going to the movies",
# #     "The Boston Celtics won the game by 20 points",
# #     "This is just a random text.",
# #     "Elden Ring is one of the best games in the last 15 years.",
# #     "L. Kornet is one of the best Celtics players.",
# #     "Larry Bird was an iconic NBA player.",
# # ]

# # Create a retriever
# retriever = Chroma.from_texts(texts, embedding=embeddings).as_retriever(
#     search_kwargs={"k": 10}
# )


In [36]:
# retriever

In [37]:
# query = "What can you tell me about the Celtics?"

# # Get relevant documents ordered by relevance score
# docs = retriever.invoke("summarize")
# docs